## Mongo Connection

Mongo connection string consists of ip and port of the server on which your mongo is running.

My mongo is running on a docker container on server ->
   * **ip:   192.168.18.4** 
   * **port:  27017**

and model requests is the name of my collection inside a db

In [18]:
from pymongo import MongoClient


connectionString = "mongodb://192.168.18.4:27017,192.168.18.4:27018,192.168.18.4:27019/magic_shop?replicaSet=mongodb-replicaset";
client = MongoClient(connectionString)

# Check Mongo
Check if mongo db is working or not with some insertions and selections.

## Insert

In [19]:
requests = [
 "I need a beer",
 "Hey! How are you Sohaib! I need 2 fresh lime",
 "Boy! Give me a pizza",
 "Hey My name is Nina! from tekken 7 I need you phone number!",
 "Hey I need Invoker's magic",
 "I want the owner of the magic shop! He just give ciggrestts to a monkey!",
 "Hey! Wanna go on a date!",
 "I need pudge arcana hook!"
]


In [83]:
from pymongo import MongoClient
from PIL import Image
import io, random
import matplotlib.pyplot as plt
from datetime import datetime



db        = client.magic_shop            # test is my database
col       = db.req                       # Here spam is my collection
brust     = 5000
count     = 0 

for i in range(0, brust):
    random_id = int(random.random()*100000)
    image= {
                'request_id': random_id,
                'request': random.choice(requests),
                'request_time':datetime.today()
                }

    image_id = col.insert_one(image).inserted_id
    count+=1
    
    
print(f"Request Done {count}")

Request Done 5000


# Getting Response for the request we processed

In [84]:

db = client.magic_shop #test is my database
col = db.req #Here spam is my collection
array = [i for i in col.find({ "request_id": random_id})]
print(array)

[{'_id': ObjectId('60e0400a0b3e8d8f57df33b4'), 'request_id': 99894, 'request': 'I need a beer', 'request_time': datetime.datetime(2021, 7, 3, 15, 46, 34, 837000)}]


# Some DataScience Stuff
As I am a data-scientist so I love to dig down the past, apply some analytical stuff, think like voker and than give recommendations.


So My plan is to 
1. get the times and see how much time do mongo take to process requests from start to end.

In [101]:
def millis_interval(end, start):
    """start and end are datetime instances"""
    diff = end - start
    millis = diff.days * 24 * 60 * 60 * 1000
    millis += diff.seconds * 1000
    millis += diff.microseconds / 1000
    return millis


processing_time = []
task_done_time  = []
not_processed   = 0

array = [i for i in col.find()]
print(len(array))
for i in array:
    
    try:
        request_time    = i['request_time']
        delivery_time   = i['processing_time']
        response_time   = i['response_time']


        processing_time.append(millis_interval(response_time , request_time))
        task_done_time.append(millis_interval(response_time , delivery_time))
    except Exception as e:
        not_processed += 1
        
print(f"Request Processed {brust - not_processed}")
print(f"Average Whole Request Processing Time this artitecture: {sum(processing_time)/len(processing_time)}")
print(f"Average Request Processed Time By CSR: {sum(task_done_time)/len(task_done_time)}")


5000
Request Processed 4878
Average Whole Request Processing Time this artitecture: 10051.783312833128
Average Request Processed Time By CSR: 1.4825748257482574


# Dropped Requests

I dont know why these requests are dropped by the artitecture thats strange. let go to stackoverflow and 

In [102]:
print(f"Total Missed Requests by the stream {( not_processed)}")

Total Missed Requests by the stream 122


# Stats


**Artitecture Stats**
    
    
| No of requests  | Time Taken | Processed Requests | missed requests | docker swarm replicas |
|-|-|-|-|-|
| 1000 | 949.51 | 996 | 4 | 1 |
| 3000 | 4387.24 | 2948 | 52 | 1 |
| 5000 | 10051.78 | 4878 | 122 | 1 |
    

# Delete All

In [103]:
for i in col.find():
    print(i['request_id'])
    col.delete_one({"request_id":i["request_id"]})

8978
39636
66462
23132
40697
82128
71411
21558
93824
49601
11660
94241
42351
79328
94976
42371
32586
61203
39667
69559
45280
68208
57208
44316
93239
57599
8385
76714
11079
67099
3071
62239
75692
26586
54789
56774
52059
34676
61508
28154
96006
9812
32912
56126
54162
55097
60328
65476
63174
12375
23771
33070
34648
36740
72647
10855
86167
71130
62908
34884
26282
70599
90387
84087
46160
15207
18866
37892
5923
85192
36527
75399
67676
10149
32477
3931
63457
77790
90738
42961
2588
78344
20413
84129
95728
4416
45334
69049
8015
719
42408
15403
51905
2366
80892
80329
81420
35995
9804
84759
2607
2009
12599
29148
21230
62197
39122
41171
684
80495
57184
80176
25748
47047
86331
70435
10580
69153
48637
77887
76036
44050
11077
88413
18251
78405
19749
33008
24519
98997
10933
32001
42150
68336
88180
46534
46893
68864
18059
17490
86088
27614
21500
45671
25338
91268
23629
28388
24175
38753
17326
12182
22972
33941
40228
29916
22369
20873
48147
7514
91234
42105
9027
41183
68741
7
92671
81395
25201
55864
687

61408
90023
27302
28531
71533
94128
77007
51522
81829
41521
95372
14596
65008
76946
43503
15094
78221
88059
67728
77234
28344
56593
13239
99128
70755
64005
82649
38105
95083
13911
91621
74416
30432
78174
2610
33956
83545
38022
19576
76587
80169
73631
26805
72493
680
26233
37433
61866
58164
8446
14133
45745
99982
76012
24380
89078
99292
8166
50623
114
43387
90641
29875
33729
42490
71887
96183
59299
6587
60494
63150
82417
3907
3414
50765
67653
71930
42454
94312
53740
75118
13469
74045
27447
88504
80594
93542
43568
42351
61352
67802
36037
96961
3303
36972
49103
31171
13946
96522
43001
71305
60933
97360
27282
68553
76689
22437
87289
69404
15905
17854
93701
19653
24770
55205
94914
5608
84155
50697
28170
15399
54392
89456
4950
80822
71777
90121
29505
5668
6336
76371
10734
22006
96018
34849
1661
35176
76030
27030
60753
47215
84213
42404
11108
43463
8613
293
93304
81227
34826
74760
97751
85131
43237
95786
68583
94407
65134
42985
22945
48063
40545
744
30690
73561
50237
21672
97918
49788
57530
6

85164
47574
38242
97428
21324
24737
11918
73418
65977
51595
96424
88978
13164
8111
43036
54615
53885
26579
86531
35169
13981
20891
65042
22628
35857
99956
37266
29073
39835
49679
93693
28508
90873
28395
19495
91535
10313
79738
80561
76874
86865
87231
58316
77378
43760
62927
69835
35464
95042
55365
72870
51403
30667
64789
77781
56624
27219
81428
13934
92668
6034
40310
4419
48958
96524
37170
20746
58004
26476
15776
75021
33370
7460
75907
78901
30391
63384
79827
27208
53521
73197
27675
20198
35166
25239
19132
66647
26273
8509
29936
35572
90230
56995
29774
90046
94446
60105
45300
49822
39343
92926
45142
14980
45450
22501
21977
40742
46361
34404
43306
20961
96062
42742
29725
52389
16178
73441
88434
33294
96462
67978
99048
62471
77350
99208
96598
5546
33330
71735
7278
97403
16294
3502
63108
50475
83241
11876
82089
56667
1947
55838
14329
87946
67699
83604
15851
14910
77857
95697
28331
22651
43248
37746
71853
48837
35054
46262
53206
16356
57879
8515
54088
76330
48534
80270
50769
94382
84119
41

19914
96285
41091
88891
21648
21701
62476
44998
15946
22722
8945
37856
58795
34646
87612
24935
88199
93963
30850
26033
81923
5689
92290
65658
42164
26908
89959
18311
40652
24913
29244
86666
97853
71314
22777
29859
67989
9416
36076
4546
25276
85717
27345
32395
22908
9866
54031
91147
5237
47940
59947
92069
85817
96334
37257
87314
62004
4164
97564
21876
49812
85170
12474
70648
48408
17391
60504
49084
6706
67638
16792
85535
78836
20518
66416
605
47740
8830
85604
94908
34029
42328
30946
82685
19824
43611
95052
50278
16371
9465
77588
45118
95933
74131
46127
41717
26611
92397
3874
64609
48431
67660
46239
10894
71518
53635
32541
61176
43595
70487
47219
32317
9047
16260
8693
31472
24910
76262
65232
74614
86717
5915
81665
24008
92028
13163
77860
50977
83769
42497
41386
30677
7426
57670
25885
27013
37413
18011
76084
75243
94321
47261
46899
69963
26693
50995
76865
81590
94348
7376
88853
21485
49163
5388
60867
21352
33669
91201
52041
57156
22612
60181
11398
25385
83257
37457
80087
31946
36086
14340